In [1]:
from helium import *
import time
import numpy as np
import pandas as pd
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import threading

# Functions To Extract Scraped Data

In [2]:
def handle_row_one_bigger(row1,row2):
    counter = len(row1) + len(row2)
    courses_of_row1 = [item.web_element.text for item in row1]
    courses_of_row2 = [item.web_element.text for item in row2]
    
    temp = pd.DataFrame({'course_code':[],'course_name':[],'section_number':[],'course_type':[],
                        'course_hours':[],'course_gender':[]})
    
    for i in range(len(courses_of_row1)-1):
        temp_lecture_df = Extracter(courses_of_row1[i])
        temp_tutorial_df = Extracter(courses_of_row2[i])
        temp = pd.concat([temp,temp_lecture_df])
        temp = pd.concat([temp,temp_tutorial_df])
    temp= temp.append(Extracter(courses_of_row1[-1]))
    return temp
        

In [3]:
def handle_row_two_bigger(row1,row2):
    counter = len(row1) + len(row2)
    courses_of_row1 = [item.web_element.text for item in row1]
    courses_of_row2 = [item.web_element.text for item in row2]
    
    temp = pd.DataFrame({'course_code':[],'course_name':[],'section_number':[],'course_type':[],
                        'course_hours':[],'course_gender':[]})
    
    for i in range(len(courses_of_row2)-1):
        temp_lecture_df = Extracter(courses_of_row1[i])
        temp_tutorial_df = Extracter(courses_of_row2[i])
        temp = pd.concat([temp,temp_tutorial_df])
        temp = pd.concat([temp,temp_lecture_df])
        
    temp= temp.append(Extracter(courses_of_row2[-1]))
    return temp
        

In [4]:
def Extracter(row):
   
    #Lecture extracter
    row=row.split(" ")
    if row[-5] == "محاضرة" or row[-5] == 'تدريب' or row[-5] == 'مشروع':
        course_code = row[0] +" "+ row[1]
        course_name = " ".join(row[2:-6])
        section_number = row[-6]
        course_type = row[-5]
        course_hours = row[-4]
        course_gender = row[-3]
        temp_df = pd.DataFrame({'course_code':[course_code],'course_name':[course_name],'section_number':[section_number],
                                'course_type':[course_type],'course_hours':[course_hours],'course_gender':[course_gender],
                               },index=[0])
        #print(temp_df)
        return temp_df
    
    elif row[-4] == 'تمارين' or row[-4] == "عملي":
        course_code = row[0] +" "+ row[1]
        course_name = " ".join(row[2:-5])
        section_number = row[-5]
        course_type = row[-4]
        course_hours = row[-3]
        course_gender = row[-2]
        temp_df  = pd.DataFrame({'course_code':[course_code],'course_name':[course_name],'section_number':[section_number],
                                'course_type':[course_type],'course_hours':[course_hours],'course_gender':[course_gender]},index=[0])
        return temp_df
    
    else:
        temp_df = pd.DataFrame({'course_code':['redundant'],'course_name':['redundant'],'section_number':['redundant'],'course_type':['redundant'],
                          'course_hours':['redundant'],'course_gender':['redundant']})
        return temp_df

In [5]:
def ExtractCourseDayAndDuration(array):
    text = array
    time_regex = re.compile(r'\d{2}:\d{2}(?: [صم])')
    timeOfCourse = []
    for i in text:
        result = re.findall(time_regex,i)
        if not len(result) == 0:
            for i in result:
                if i not in timeOfCourse:
                    timeOfCourse.append(i)
                
    return timeOfCourse
    

In [6]:
def ExtractCourseDay(text):
    digits_only = re.findall('\d', text)
    return digits_only

# Scrape DataFrame for saving and cleaning later

In [7]:
courses_df = pd.DataFrame({'course_code':[],'course_name':[],'section_number':[],'course_type':[],
                          'course_hours':[],'course_gender':[]})

In [8]:
finals_df = pd.DataFrame({'course_day':[],'course_time':[],'final_exam_day':[],
                          'final_exam_date':[],'final_exam_time':[],'lecturer':[]})

In [9]:
start_chrome()
go_to("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")
#click("أكاديمي")
#time.sleep(1)
#click("المقررات المطروحة")
#time.sleep(1)
#click("أكاديمي")
time.sleep(1)
click(Link("المقررات المطروحة",above="المقررات المطروحة - بحث متقدم"))
time.sleep(1)
click("علوم الحاسب والمعلومات")
time.sleep(1)
click("هندسة البرمجيات")
time.sleep(1)
click("SWEN-هندسة البرمجيات")
time.sleep(1)
click("إبحث")

counter = 1
while counter < 54:
    
    # Get all of the courses 
    time.sleep(1)
    row1 = find_all(S('.ROW1'))
    row2 = find_all(S('.ROW2'))
    time.sleep(1)
    if len(row1) > len(row2):
        courses_df = courses_df.append(handle_row_one_bigger(row1,row2))
    
    else:
        courses_df = courses_df.append(handle_row_two_bigger(row1,row2))
    
    # Get Final Exam, Course Day and Course Time
    #time.sleep(1)
    #links = find_all((Link("التفاصيل")))
    #time.sleep(1)
    #pattern = r"\d+"
    #first = str(links[0])
    # find all occurrences of the pattern in the text
    #firstInList = int(re.findall(pattern, first)[0])
    #second = str(links[-1])
    #secondInList = int(re.findall(pattern,second)[0])
    # print the matches
    """"
    row = None
    if firstInList>secondInList:
        for i in reversed(links):
            time.sleep(1)
            click(i)
            time.sleep(1)
            courseDay = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
            courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
            courseDay = [item.web_element.text for item in courseDay]
            courseTime = [item.web_element.text for item in courseTime]
            #instructorDiv
            #courseDay = ExtractCourseDay(str(row[0]))
            #courseTime = ExtractCourseDayAndDuration(row)
            examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
            examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
            examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
            lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]
            temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
            'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]],
                                   'lecturer':[lecturer]})
        
            time.sleep(1)
            finals_df = pd.concat([finals_df,temp_df])
            click(S(".ui-icon-close"))
        
    else:
        
        for i in links:
            time.sleep(1)
            click(i)
            time.sleep(1)

            courseDay = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
            courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
            courseDay = [item.web_element.text for item in courseDay]
            courseTime = [item.web_element.text for item in courseTime]
            examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
            examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
            examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
            lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]
            temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
            'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]],
                                   'lecturer':[lecturer]})

            time.sleep(1)
            finals_df = pd.concat([finals_df,temp_df])
            click(S(".ui-icon-close"))
    """
    row = None
        
    counter +=1
    row1 = []
    row2  = []
    click(S(".pui-paginator-next"))


In [10]:
courses_df

,course_code,course_name,section_number,course_type,course_hours,course_gender
0,254 ريض,الطرائق العددية,5582,محاضرة,3,ذكر
0,254 ريض,الطرائق العددية,20499,تمارين,ذكر,مفتوحة
0,254 ريض,الطرائق العددية,81601,محاضرة,3,ذكر
0,254 ريض,الطرائق العددية,81602,تمارين,ذكر,مفتوحة
0,254 ريض,الطرائق العددية,81603,محاضرة,3,ذكر
...,...,...,...,...,...,...
0,100 عرب,مهارات الكتابة.,85547,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76566,محاضرة,2,ذكر
0,479 هاب,التدريب الميداني,73343,تدريب,1,ذكر
0,100 عرب,مهارات الكتابة.,85568,محاضرة,2,ذكر


In [11]:
start_chrome()
go_to("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")
#click("أكاديمي")
#time.sleep(1)
#click("المقررات المطروحة")
#time.sleep(1)
#click("أكاديمي")
#time.sleep(1)
click(Link("المقررات المطروحة",above="المقررات المطروحة - بحث متقدم"))
time.sleep(1)
click("علوم الحاسب والمعلومات")
time.sleep(1)
click("هندسة البرمجيات")
time.sleep(1)
click("SWEN-هندسة البرمجيات")
time.sleep(1)
click("إبحث")
counter = 1
while counter < 54:
    
    # Get Final Exam, Course Day and Course Time
    time.sleep(1)
    links = find_all((Link("التفاصيل")))
    time.sleep(1)
    pattern = r"\d+"
    first = str(links[0])
    # find all occurrences of the pattern in the text
    firstInList = int(re.findall(pattern, first)[0])
    second = str(links[-1])
    secondInList = int(re.findall(pattern,second)[0])
    #print the matches
    
    row = None
    if firstInList>secondInList:
        for i in reversed(links):
            time.sleep(1)
            click(i)
            time.sleep(1)
            courseDay = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
            courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
            courseDay = [item.web_element.text for item in courseDay]
            courseTime = [item.web_element.text for item in courseTime]
            #instructorDiv
            examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
            examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
            examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
            lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]
            temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
            'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]],
                                   'lecturer':[lecturer]})
        
            time.sleep(1)
            finals_df = pd.concat([finals_df,temp_df])
            click(S(".ui-icon-close"))
        
    else:
        
        for i in links:
            time.sleep(1)
            click(i)
            time.sleep(1)
            courseDay = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
            courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
            courseDay = [item.web_element.text for item in courseDay]
            courseTime = [item.web_element.text for item in courseTime]
            examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
            examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
            examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
            lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]
            temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
            'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]],
                                   'lecturer':[lecturer]})

            time.sleep(1)
            finals_df = pd.concat([finals_df,temp_df])
            click(S(".ui-icon-close"))
    
    row = None
        
    counter +=1
    row1 = []
    row2  = []
    click(S(".pui-paginator-next"))


In [24]:
courses_df.tail(20)

x = courses_df[:-9]
x

,course_code,course_name,section_number,course_type,course_hours,course_gender
0,254 ريض,الطرائق العددية,5582,محاضرة,3,ذكر
0,254 ريض,الطرائق العددية,20499,تمارين,ذكر,مفتوحة
0,254 ريض,الطرائق العددية,81601,محاضرة,3,ذكر
0,254 ريض,الطرائق العددية,81602,تمارين,ذكر,مفتوحة
0,254 ريض,الطرائق العددية,81603,محاضرة,3,ذكر
...,...,...,...,...,...,...
0,100 عرب,مهارات الكتابة.,76553,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76555,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76556,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76558,محاضرة,2,ذكر


In [27]:
y = finals_df[:-10]
y

,course_day,course_time,final_exam_day,final_exam_date,final_exam_time,lecturer
0,[1 3 5],[ 08:00 ص - 09:05 ص],5,1444-12-04,01:00 م - 04:00 م,[رضوان بوت]
0,"[1, 1]","[ 06:30 م - 07:35 م, 07:45 م - 08:50 م]",---,---,---,[لم يحدد من الكلية]
0,[1 3 5],[ 09:15 ص - 10:20 ص],5,1444-12-04,01:00 م - 04:00 م,[رضوان بوت]
0,"[1, 1]","[ 01:30 م - 02:35 م, 02:45 م - 03:50 م]",---,---,---,[لم يحدد من الكلية]
0,[1 3 5],[ 10:30 ص - 11:35 ص],5,1444-12-04,01:00 م - 04:00 م,[محمد بن حمده عبدالواحد]
...,...,...,...,...,...,...
0,"[4, 4]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[2, 2]","[ 04:00 م - 05:05 م, 05:15 م - 06:20 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[1, 1]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[3, 3]","[ 01:30 م - 02:35 م, 02:45 م - 03:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]


In [38]:
z=pd.concat([x,y],axis=1)
z.query("section_number == '86289'")

,course_code,course_name,section_number,course_type,course_hours,course_gender,course_day,course_time,final_exam_day,final_exam_date,final_exam_time,lecturer
0,318 هال,النظم المضمنة,86289,عملي,ذكر,مغلقة,"[4, 4]","[ 08:00 ص - 09:05 ص, 09:15 ص - 10:20 ص]",---,---,---,[لم يحدد من الكلية]


In [40]:
z.to_excel('swe_combined.xlsx')

In [22]:
finals_df.tail(20)

,course_day,course_time,final_exam_day,final_exam_date,final_exam_time,lecturer
0,"[5, 5]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[3, 3]","[ 01:30 م - 02:35 م, 02:45 م - 03:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[4, 4]","[ 05:15 م - 06:20 م, 06:30 م - 07:35 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[4, 4]","[ 08:00 ص - 09:05 ص, 09:15 ص - 10:20 ص]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[3, 3]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[4, 4]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[2, 2]","[ 04:00 م - 05:05 م, 05:15 م - 06:20 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[1, 1]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[3, 3]","[ 01:30 م - 02:35 م, 02:45 م - 03:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]
0,"[4, 4]","[ 10:30 ص - 11:35 ص, 11:45 ص - 12:50 م]",4,1444-11-18,04:00 م - 06:00 م,[لم يحدد من الكلية]


In [146]:
courses_df.tail(10)

,course_code,course_name,section_number,course_type,course_hours,course_gender
0,100 عرب,مهارات الكتابة.,76560,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76563,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76562,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76565,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76564,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,85547,محاضرة,2,ذكر
0,100 عرب,مهارات الكتابة.,76566,محاضرة,2,ذكر
0,479 هاب,التدريب الميداني,73343,تدريب,1,ذكر
0,100 عرب,مهارات الكتابة.,85568,محاضرة,2,ذكر
0,479 هاب,التدريب الميداني,81883,تدريب,1,ذكر


In [114]:
courses_temp = courses_df.reset_index()
courses_temp = courses_temp.drop('index',axis=1)

In [115]:
finals_temp = finals_df.reset_index()
finals_temp = finals_temp.drop('index',axis=1)

In [117]:
courses_finals_merged = courses_temp.join(finals_temp)

In [118]:
courses_finals_merged.to_excel('finals_courses_merged.xlsx',index=False)

In [62]:
redundantCourses = ['101 نهج','100 عرب','201 حسب','101 ادا']

In [68]:
courses_finals_df = pd.concat([courses_df,finals_df])

In [74]:
courses_df

,course_code,course_name,section_number,course_type,course_hours,course_gender,lecturer
0,254 ريض,الطرائق العددية,5582,محاضرة,3,ذكر,NaN
0,254 ريض,الطرائق العددية,20499,تمارين,ذكر,مفتوحة,NaN
0,254 ريض,الطرائق العددية,81601,محاضرة,3,ذكر,NaN
0,254 ريض,الطرائق العددية,81602,تمارين,ذكر,مفتوحة,NaN
0,254 ريض,الطرائق العددية,81603,محاضرة,3,ذكر,NaN
...,...,...,...,...,...,...,...
0,100 عرب,مهارات الكتابة.,85547,محاضرة,2,ذكر,NaN
0,100 عرب,مهارات الكتابة.,76566,محاضرة,2,ذكر,NaN
0,479 هاب,التدريب الميداني,73343,تدريب,1,ذكر,NaN
0,100 عرب,مهارات الكتابة.,85568,محاضرة,2,ذكر,NaN


In [203]:
finals_df

,course_day,course_time,final_exam_day,final_exam_date,final_exam_time,lecturer


In [69]:
temp = courses_finals_df.query(f"course_code == '{redundantCourses[3]}'").section_number

In [71]:
mask = courses_finals_df.course_code.isin(redundantCourses)

In [72]:
courses_finals_df[~mask]

,course_code,course_name,section_number,course_type,course_hours,course_gender,lecturer,course_day,course_time,final_exam_day,final_exam_date,final_exam_time
0,254 ريض,الطرائق العددية,5582,محاضرة,3,ذكر,NaN,NaN,NaN,NaN,NaN,NaN
0,254 ريض,الطرائق العددية,20499,تمارين,ذكر,مفتوحة,NaN,NaN,NaN,NaN,NaN,NaN
0,254 ريض,الطرائق العددية,81601,محاضرة,3,ذكر,NaN,NaN,NaN,NaN,NaN,NaN
0,254 ريض,الطرائق العددية,81602,تمارين,ذكر,مفتوحة,NaN,NaN,NaN,NaN,NaN,NaN
0,254 ريض,الطرائق العددية,81603,محاضرة,3,ذكر,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
0,318 هال,النظم المضمنة,86287,عملي,ذكر,مفتوحة,NaN,NaN,NaN,NaN,NaN,NaN
0,318 هال,النظم المضمنة,86288,تمارين,ذكر,مفتوحة,NaN,NaN,NaN,NaN,NaN,NaN
0,318 هال,النظم المضمنة,86289,عملي,ذكر,مغلقة,NaN,NaN,NaN,NaN,NaN,NaN
0,479 هاب,التدريب الميداني,73343,تدريب,1,ذكر,NaN,NaN,NaN,NaN,NaN,NaN


# Extract Final Exam Date / Course Time/ Coure days

In [119]:
from helium import *
import selenium
import time
start_chrome()
go_to("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")
#click("أكاديمي")
#time.sleep(1)
#click("المقررات المطروحة")
#time.sleep(1)
#click("أكاديمي")
#time.sleep(1)
click(Link("المقررات المطروحة",above="المقررات المطروحة - بحث متقدم"))
time.sleep(1)
click("علوم الحاسب والمعلومات")
time.sleep(1)
click("هندسة البرمجيات")
time.sleep(1)
click("SWEN-هندسة البرمجيات")
time.sleep(1)
click("إبحث")
time.sleep(1)


for i in range(35):
    click(S(".pui-paginator-next"))
    
#click(S(".pui-paginator-next"))
time.sleep(1)
row1 = find_all(S('.ROW1'))
row2 = find_all(S('.ROW2'))
#links = find_all((Link("التفاصيل")))
#time.sleep(1)
#click(links[11])
#time.sleep(1)
"""
courseDay =find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='اليوم'))
courseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below='الوقت'))
courseDay = [item.web_element.text for item in courseDay]
courseTime = [item.web_element.text for item in courseTime]
lecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]


examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]
temp_df = pd.DataFrame({'course_day':[courseDay],'course_time':[courseTime],
    'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]]})
"""    
    #time.sleep(1)
    #finals_df = pd.concat([finals_df,temp_df])
    #click(S(".ui-icon-close"))
    #row = None
    #counter +=1
    #click(S(".pui-paginator-next"))




'\ncourseDay =find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below=\'اليوم\'))\ncourseTime = find_all(S("#sectionDiv > table > tbody > tr > td > nobr",below=\'الوقت\'))\ncourseDay = [item.web_element.text for item in courseDay]\ncourseTime = [item.web_element.text for item in courseTime]\nlecturer = [item.web_element.text for item in find_all(S("#instructorDiv"))]\n\n\nexamDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]\nexamTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]\nexamDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]\ntemp_df = pd.DataFrame({\'course_day\':[courseDay],\'course_time\':[courseTime],\n    \'final_exam_day\':[examDay[0]], \'final_exam_date\':[examDate[0]],\'final_exam_time\':[examTime[0]]})\n'

In [135]:
row2[0].web_element.text.split("   ")

['497 هاب مشروع تخرج -2 33644 مشروع 3 ذكر مفتوحة التفاصيل']

In [124]:
temp_df

,course_day,course_time,final_exam_day,final_exam_date,final_exam_time
0,[1 3 5],[ 08:00 ص - 09:05 ص],1,1444-11-22,01:00 م - 04:00 م


In [30]:
row1

['2 4', '4']

In [29]:
row2

['  10:30 ص - 11:35 ص', '  11:45 ص - 12:50 م']

In [380]:
concatenated_str = ' '.join(set(row1[0]).union(set(row2[0])))

In [381]:
concatenated_str

'3 1  '

In [350]:
row2[0]

'1 3'

In [ ]:
pattern = r''

In [206]:
finals_df.iloc[0,1][0]

'0'

# Handling Classes with 2-4 days in course not 1-3-5

In [69]:
from helium import *
import selenium
import time
start_chrome()
go_to("https://edugate.ksu.edu.sa/ksu/ui/guest/timetable/index/mainScheduleTreeCoursesIndex.faces")
#click("أكاديمي")
#time.sleep(1)
#click("المقررات المطروحة")
#time.sleep(1)
#click("أكاديمي")
#time.sleep(1)
click(Link("المقررات المطروحة",above="المقررات المطروحة - بحث متقدم"))
time.sleep(1)
click("علوم الحاسب والمعلومات")
time.sleep(1)
click("هندسة البرمجيات")
time.sleep(1)
click("SWEN-هندسة البرمجيات")
time.sleep(1)
click("إبحث")
time.sleep(1)
click(S(".pui-paginator-next"))
time.sleep(1)
links = None   
links = find_all((Link("التفاصيل",below="التفاصيل")))
print(links)
courseDetails = None
time.sleep(1)
courseDetails = find_all(S("#sectionDiv"))
courseDetails = [item.web_element.text for item in courseDetails]
#print(courseDetails)
#examDay = [item.web_element.text for item in find_all(S("#examDayDiv"))]
#examTime = [item.web_element.text for item in find_all(S("#examTimeDiv"))]
#examDate = [item.web_element.text for item in find_all(S("#examDateDiv"))]

click(S(".pui-paginator-next"))
time.sleep(1)
links = find_all((Link("التفاصيل")))

#temp_df = pd.DataFrame({'course_day':[row[0]],'course_time':[row[1]],
                      #  'final_exam_day':[examDay[0]], 'final_exam_date':[examDate[0]],'final_exam_time':[examTime[0]]})


[<a href="" onclick="javascript:showToolTipDilaog(event,'29');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'28');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'27');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'26');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'25');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'24');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'23');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'22');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'21');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'20');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'19');return false;">...</a>, <a href="" onclick="javascript:showToolTipDilaog(event,'18');ret

In [72]:
courseDetails

[]

In [87]:
for i in reversed(links):

التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل
التفاصيل


In [213]:
for i in text:
    print(i)

اليوم الوقت القاعة
5
01:30 م - 02:35 م
0140 04 1 B 024
5
02:45 م - 03:50 م
0140 04 1 B 024


In [203]:
text = ['اليوم الوقت القاعة\n5', '01:30 م - 02:35 م', '0140 04 1 B 024\n5', '02:45 م - 03:50 م', '0140 04 1 B 024']
time_regex = re.compile(r'\d{2}:\d{2}(?: [صم])')
timeOfCourse = []
for i in text:
    result = re.findall(time_regex,i)
    if len(result) == 0:
        print('no time was here')
    
    else:
        for i in result:
            if i not in timeOfCourse:
                timeOfCourse.append(i)

no time was here
no time was here
no time was here


In [197]:
text = ['اليوم الوقت القاعة\n2', '08:00 ص - 09:05 ص', '0140 04 1 A 072\n2', '09:15 ص - 10:20 ص', '0140 04 1 A 072\n4', '09:15 ص - 10:20 ص', '0140 04 1 A 075']
time_regex = re.compile(r'\d{2}:\d{2}(?: [صم])')
timeOfCourse = []
for i in text:
    result = re.findall(time_regex,i)
    if len(result) == 0:
        print('no time was here')
    
    else:
        for i in result:
            if i not in timeOfCourse:
                timeOfCourse.append(i)
                

no time was here
no time was here
no time was here
no time was here


In [195]:
whatIsTime = ExtractCourseDayAndDuration(text)

In [196]:
whatIsTime

['08:00 ص', '10:20 ص']

In [198]:
timeOfCourse

['08:00 ص', '09:05 ص', '09:15 ص', '10:20 ص']